<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/fraud/mockup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/opd_discharge_August_2024.csv')
data = data[data['order_item']!='[]']
df = data.copy()
data.shape

(16477, 92)

## extract icd10 func

In [4]:
import json

def extract_primary_icd10(diagnosis_str):
    try:
        diagnoses = json.loads(diagnosis_str)
        if isinstance(diagnoses, list):
            for d in diagnoses:
                if isinstance(d, dict) and d.get("dx_type") == "PP":
                    return d.get("icd10")
    except (json.JSONDecodeError, TypeError):
        pass
    return None

data['icd10'] = data['diagnosis'].apply(extract_primary_icd10)

In [5]:
icd10_df = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/icd10 - ICD-10 v.2019.csv')

In [6]:
print(f'โรคทั้งหมดมี unique จำนวน', icd10_df['ICD-10 Code'].nunique(), 'โรค')
icd10_df[['ICD-10 Code','Descriptions (EN)']].head()

โรคทั้งหมดมี unique จำนวน 38385 โรค


,ICD-10 Code,Descriptions (EN)
0,A000,"Cholera due to Vibrio cholerae 01, biovar chol..."
1,A001,"Cholera due to Vibrio cholerae 01, biovar eltor"
2,A009,"Cholera, unspecified"
3,A010,Typhoid fever
4,A011,Paratyphoid fever A


## ราคากลางของ billing แบ่งตามโรค

In [35]:
import json
import pandas as pd

# สร้าง DataFrame จาก data
focus = pd.DataFrame(data)

# คัดลอกข้อมูลและแปลงให้การใช้ icd10 ที่ extract มาแล้ว
focus['order_item'] = focus['order_item'].apply(lambda x: json.loads(x))  # แปลง order_item เป็น JSON

all_billing_summary = []

# ดึงรายการรหัส ICD-10 ที่ไม่ซ้ำกันจาก icd10 ที่ extract มาแล้ว
unique_icd10_codes = focus['icd10'].unique()

for icd10 in unique_icd10_codes:
    # กรองข้อมูลสำหรับ ICD-10 รหัสนั้น
    filtered_data = focus[focus['icd10'] == icd10]

    # ตรวจสอบว่า filtered_data มีจำนวนแถวมากกว่าหนึ่งแถว
    if icd10 == 'W260' and filtered_data.shape[0] > 1:
        print(f"ข้อมูลที่กรองสำหรับ ICD-10 {icd10} มีหลายแถว: {filtered_data.shape[0]}")

    # ตรวจหาค่า illness_type ที่พบบ่อยที่สุดสำหรับรหัส icd10 นั้น (หากมีหลายประเภท)
    illness_type = filtered_data['illnes_type'].mode().iloc[0] if not filtered_data['illnes_type'].mode().empty else None

    # สร้างรายการ billing
    billing_items = []
    for items in filtered_data['billing']:
        items = json.loads(items)
        for item in items:
            billing_items.append(item)

    billing_items_df = pd.DataFrame(billing_items)
    billing_items_df['billing_initial'] = billing_items_df['billing_initial'].astype(float)

    # ลบข้อมูลซ้ำจาก billing_items_df เพื่อป้องกันไม่ให้ข้อมูลซ้ำในตาราง
    billing_items_df.drop_duplicates(inplace=True)

    # คำนวณ summary ของ billing ตาม billing code และชื่อ
    billing_summary = billing_items_df.groupby(['simb_billing_code', 'local_billing_name']).agg(
        average_initial_price=('billing_initial', 'median'),
        billing_count=('billing_initial', 'count'),
        min_initial_price=('billing_initial', 'min'),
        max_initial_price=('billing_initial', 'max'),
        price_list=('billing_initial', lambda x: sorted(x.unique().tolist()))  # สร้าง list ของราคาที่ไม่ซ้ำ
    ).reset_index()

    # แปลง price_list เป็น string ก่อนการใช้ drop_duplicates
    billing_summary['price_list'] = billing_summary['price_list'].apply(lambda x: str(x))

    # คำนวณ range ของราคาตาม min-max
    billing_summary['range'] = billing_summary['min_initial_price'].astype(str) + '-' + billing_summary['max_initial_price'].astype(str)

    # เพิ่มคอลัมน์ icd10 และ illness_type
    billing_summary['icd10'] = icd10
    billing_summary['illness_type'] = illness_type

    # ลบข้อมูลซ้ำใน billing_summary ก่อนเพิ่มเข้าไปใน all_billing_summary
    billing_summary.drop_duplicates(inplace=True)

    # เพิ่มข้อมูล billing_summary เข้าในรายการ all_billing_summary
    all_billing_summary.append(billing_summary)

# รวมข้อมูลทั้งหมดใน all_billing_summary เข้าด้วยกัน
final_summary = pd.concat(all_billing_summary, ignore_index=True)

# ลบข้อมูลซ้ำใน final_summary
final_summary.drop_duplicates(inplace=True)

# แสดงตารางผลลัพธ์
display(final_summary)


,simb_billing_code,local_billing_name,average_initial_price,billing_count,min_initial_price,max_initial_price,price_list,range,icd10,illness_type
0,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,1032.0,17,0.0,3581.0,"[0.0, 68.0, 100.0, 117.0, 170.0, 490.0, 1032.0...",0.0-3581.0,S711,ACC
1,1.1.1(4),ค่าสารอาหารทางเส้นเลือด,116.0,1,116.0,116.0,[116.0],116.0-116.0,S711,ACC
2,1.1.12,Nursing and Midwifery Charge ค่าบริการพยาบาล,150.0,9,60.0,650.0,"[60.0, 80.0, 100.0, 140.0, 150.0, 175.0, 185.0...",60.0-650.0,S711,ACC
3,1.1.14(1),Packaged Medical Charges ค่าบริการชุดเหมาจ่าย,250.0,4,100.0,300.0,"[100.0, 200.0, 300.0]",100.0-300.0,S711,ACC
4,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,175.0,11,50.0,350.0,"[50.0, 80.0, 120.0, 150.0, 175.0, 185.0, 310.0...",50.0-350.0,S711,ACC
...,...,...,...,...,...,...,...,...,...,...
12473,1.2.1,Physician Evaluation and Management Services,600.0,1,600.0,600.0,[600.0],600.0-600.0,K583,ILL
12474,1.1.12,Nursing and Midwifery Charge ค่าบริการพยาบาล,140.0,1,140.0,140.0,[140.0],140.0-140.0,M7917,ILL
12475,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,120.0,1,120.0,120.0,[120.0],120.0-120.0,M7917,ILL
12476,1.2.1,Physician Evaluation and Management Services,500.0,1,500.0,500.0,[500.0],500.0-500.0,M7917,ILL


In [41]:
final_summary.to_csv('/content/drive/MyDrive/AIEngineer/fraud/mockup/range_billing_by_icd10.csv', index=False)

In [12]:
# bill = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/mockup/range_billing_by_icd10.csv')
# item = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/mockup/range_order_by_icd10.csv')

In [36]:
json.loads(data[data['icd10']=='W260'][['billing','total_bill_amount','diagnosis','icd10','order_item']]['billing'].iloc[0])

[{'billing_initial': '70',
  'billing_discount': '0',
  'simb_billing_code': '1.1.2(1)',
  'billing_net_amount': '70.00',
  'local_billing_code': 'SUP1',
  'local_billing_name': 'Medical Supplies 1   เวชภัณฑ์1'},
 {'billing_initial': '300',
  'billing_discount': '0',
  'simb_billing_code': '1.1.14(1)',
  'billing_net_amount': '300.00',
  'local_billing_code': 'DRES',
  'local_billing_name': 'Packaged Medical Charges  ค่าบริการชุดเหมาจ่าย '}]

In [37]:
data[data['icd10']=='W260']

,id,seq,created_at,updated_at,deleted_at,service_setting,illnes_type,policy_type,cid,cid_type,...,remark,created_submit_by,updated_submit_by,created_submit_at,updated_submit_at,other_insurer,is_check_claim_balance,is_reimbursement,epayment_id,icd10
39220,05103068-a9f1-4249-954c-d9bea6f09fef,1724226240122879309,2024-08-21 14:44:00.122 +0700,2024-09-01 01:03:12.063 +0700,NaN,OPD,ILL,CS,3130100511451,NATIONAL_ID,...,NaN,NaN,NaN,NaN,NaN,False,NaN,N,fd5fbb30-9770-44d0-acdc-e05726cc9d9c,W260


In [39]:
final_summary[final_summary['icd10']=='W260']

,simb_billing_code,local_billing_name,average_initial_price,billing_count,min_initial_price,max_initial_price,price_list,range,icd10,illness_type
11382,1.1.14(1),Packaged Medical Charges ค่าบริการชุดเหมาจ่าย,300.0,1,300.0,300.0,[300.0],300.0-300.0,W260,ILL
11383,1.1.2(1),Medical Supplies 1 เวชภัณฑ์1,70.0,1,70.0,70.0,[70.0],70.0-70.0,W260,ILL


## ราคากลางของ order_item แบ่งตามโรค

In [42]:
import json
import pandas as pd

# สร้าง DataFrame จาก data
focus = pd.DataFrame(data)

# แปลงคอลัมน์ order_item ให้เป็น JSON
focus['order_item'] = focus['order_item'].apply(lambda x: json.loads(x))  # แปลง order_item เป็น JSON

all_billing_summary = []

# ดึงรายการรหัส ICD-10 ที่ไม่ซ้ำกันจาก icd10 ที่ extract มาแล้ว
unique_icd10_codes = focus['icd10'].unique()

for icd10 in unique_icd10_codes:
    # กรองข้อมูลสำหรับ ICD-10 รหัสนั้น
    filtered_data = focus[focus['icd10'] == icd10]

    # สร้างรายการ order_item
    order_items = []
    for items in filtered_data['order_item']:
        for item in items:
            # ตรวจสอบว่ามีคีย์ 'initial' หรือไม่
            if 'initial' in item:
                order_items.append(item)

    # สร้าง DataFrame จากรายการ order_items
    order_items_df = pd.DataFrame(order_items)

    # ถ้าคอลัมน์ 'initial' มีอยู่ก็สามารถแปลงเป็น float ได้
    if 'initial' in order_items_df.columns:
        order_items_df['initial'] = order_items_df['initial'].astype(float)

        # คำนวณ summary ของ order_item ตาม billing code และชื่อ
        order_summary = order_items_df.groupby(['simb_billing_code', 'local_billing_name', 'item_id', 'item_name']).agg(
            average_initial_price=('initial', 'median'),
            order_count=('initial', 'count'),
            min_initial_price=('initial', 'min'),
            max_initial_price=('initial', 'max'),
            price_list=('initial', lambda x: sorted(x.unique().tolist()))  # สร้าง list ของราคาที่ไม่ซ้ำกัน
        ).reset_index()

        # เพิ่มคอลัมน์ icd10
        order_summary['icd10'] = icd10

        # คำนวณ range ของราคาตาม min-max
        order_summary['range'] = order_summary['min_initial_price'].astype(str) + '-' + order_summary['max_initial_price'].astype(str)

        # เพิ่มข้อมูล order_summary เข้าในรายการ all_billing_summary
        all_billing_summary.append(order_summary)

# รวมข้อมูลทั้งหมดใน all_billing_summary เข้าด้วยกัน
final_order_summary = pd.concat(all_billing_summary, ignore_index=True)

# แสดงตารางผลลัพธ์
display(final_order_summary)

,simb_billing_code,local_billing_name,item_id,item_name,average_initial_price,order_count,min_initial_price,max_initial_price,price_list,icd10,range
0,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,0101H03R0020O01,= ยาเม็ด = Reparil 20 mg #,445.0,1,445.0,445.0,[445.0],S711,445.0-445.0
1,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,0104B04T0500O07,Tylenol 650 mg แบบออกฤทธิ์ 8 ชั่วโมง #,281.5,2,188.0,375.0,"[188.0, 375.0]",S711,188.0-375.0
2,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,0104B06T0315L01,TWC (Para300mg+Codeine15mg) (กส),195.0,1,195.0,195.0,[195.0],S711,195.0-195.0
3,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,0104C02A0090O04,Arcoxia 90 mg #,840.0,2,560.0,1120.0,"[560.0, 1120.0]",S711,560.0-1120.0
4,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,0104C06C0200O01,Celebrex 200 mg #,933.5,2,778.0,1089.0,"[778.0, 1089.0]",S711,778.0-1089.0
...,...,...,...,...,...,...,...,...,...,...,...
58957,1.2.1,Physician Evaluation and Management Services,DRFEE,ค่าแพทย์,600.0,1,600.0,600.0,[600.0],K583,600.0-600.0
58958,1.1.12,Nursing and Midwifery Charge ค่าบริการพยาบาล,REGFEE2,ค่าบริการพยาบาล,140.0,1,140.0,140.0,[140.0],M7917,140.0-140.0
58959,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,REGFEE1,ค่าบริการทางการแพทย์อื่นๆ,120.0,1,120.0,120.0,[120.0],M7917,120.0-120.0
58960,1.2.1,Physician Evaluation and Management Services,DRFEE,ค่าแพทย์,500.0,1,500.0,500.0,[500.0],M7917,500.0-500.0


In [43]:
print(final_order_summary['icd10'].nunique())
print(final_summary['icd10'].nunique())

1883
1883


In [46]:
final_order_summary.to_csv('/content/drive/MyDrive/AIEngineer/fraud/mockup/range_order_by_icd10.csv', index=False)

## check

In [51]:
data[data['icd10']=='W260'][['order_item','billing','icd10','total_bill_amount']]

,order_item,billing,icd10,total_bill_amount
39220,"[{""initial"": ""70"", ""item_id"": ""SUP1"", ""discoun...","[{""billing_initial"": ""70"", ""billing_discount"":...",W260,370.0


In [47]:
final_order_summary[final_order_summary['icd10']=='W260']

,simb_billing_code,local_billing_name,item_id,item_name,average_initial_price,order_count,min_initial_price,max_initial_price,price_list,icd10,range
57096,1.1.14(1),Packaged Medical Charges ค่าบริการชุดเหมาจ่าย,DRES,DRESSING กลาง[OPD][,300.0,1,300.0,300.0,[300.0],W260,300.0-300.0
57097,1.1.2(1),Medical Supplies 1 เวชภัณฑ์1,SUP1,EASI - FIX COHESIVE,70.0,1,70.0,70.0,[70.0],W260,70.0-70.0


In [48]:
final_summary[final_summary['icd10']=='W260']

,simb_billing_code,local_billing_name,average_initial_price,billing_count,min_initial_price,max_initial_price,price_list,range,icd10,illness_type
11382,1.1.14(1),Packaged Medical Charges ค่าบริการชุดเหมาจ่าย,300.0,1,300.0,300.0,[300.0],300.0-300.0,W260,ILL
11383,1.1.2(1),Medical Supplies 1 เวชภัณฑ์1,70.0,1,70.0,70.0,[70.0],70.0-70.0,W260,ILL


## **mockup data**
>

> **diagnosis + billing**
1. **เลือกโรค (ICD-10)**: สุ่มโรคจากตาราง icd10_df เพื่อใส่ลงในข้อมูล mockup ของแต่ละแถว โดยในแต่ละโรคจะมีรายการ billing ที่เหมาะสมตามตาราง final_summary
2. **สร้างรายการ billing**: ใช้ข้อมูลใน final_summary เพื่อกำหนดช่วงราคาของแต่ละรายการใน billing โดยสุ่มราคาในช่วงที่กำหนด (range) สำหรับรายการนั้นๆ

In [52]:
# Sample data for each field
hospital_codes = [11787, 40765, 11583, 11558, 11768, 23781, 41829, 11864, 14683,
                  21673, 11720, 11771, 14591, 11543, 41345, 24258, 11963, 24257,
                  13766, 11645, 14354, 14550, 11766, 23924, 11878, 12026, 41556,
                  12188, 12226, 12159, 13779, 41674, 14641, 14534, 12139, 11765,
                  28875, 42025, 41590, 23875, 11919, 11730, 11537, 10814, 41569,
                  11845, 11708, 11644, 11648, 10686, 44109, 10827, 10734, 10663,
                  10679, 42167, 11874, 42770, 10821, 44353, 42015, 11037, 11656,
                  10695, 12230, 42141, 10849, 11359, 11621, 12212, 13778, 11920,
                  10674, 10723, 10671, 10998, 10822, 10667, 11360, 10832]

# Filter ICD-10 codes based on final_summary
icd10_codes_in_summary = final_summary['icd10'].unique()
diagnosis_samples = [
    {'icd10': row['ICD-10 Code'], 'dx_name': row['Descriptions (EN)'], 'dx_type': 'PP'}
    for _, row in icd10_df.iterrows()
    if row['ICD-10 Code'] in icd10_codes_in_summary
]

In [53]:
import pandas as pd
import random
from faker import Faker
import json

# Initial setup
fake = Faker(['en_US', 'th_TH'])
num_rows = 5000

# Generate mock data
data = []
for _ in range(num_rows):
    # Basic patient and doctor info
    hospital_code = random.choice(hospital_codes)
    cid = fake.unique.random_number(digits=13)
    patient = {
        'hn': f"{random.randint(10, 99)}-{random.randint(10000, 99999)}",
        'dob': fake.date_of_birth().isoformat(),
        'gender': random.choice(['M', 'F']),
        'last_name': fake.last_name(),
        'first_name': fake.first_name(),
        'title_name': random.choice(['นาย', 'นาง', 'นางสาว']),
        'patient_registration_id': fake.uuid4()
    }
    vital_sign = [{
        'heart_rate': str(random.randint(60, 100)),
        'pain_score': str(random.randint(0, 10)),
        'systolic_bp': f"{random.uniform(90, 180):.2f}",
        'temperature': f"{random.uniform(36.0, 37.5):.2f}",
        'diastolic_bp': f"{random.uniform(60, 120):.2f}",
        'respiratory_rate': str(random.randint(12, 20)),
        'oxygen_saturation': f"{random.uniform(95, 100):.2f}",
        'vital_sign_entry_datetime': fake.date_time_this_year().strftime("%Y-%m-%d %H:%M")
    }]
    doctor = [{
        'role': 'OWNER',
        'title': 'นพ.',
        'license': str(random.randint(10000, 99999)),
        'lastname': fake.last_name(),
        'firstname': fake.first_name()
    }]

    # Select a random diagnosis ICD-10
    diagnosis = [random.choice(diagnosis_samples)]
    diag = diagnosis[0]

    # Retrieve illness_type for the selected diagnosis ICD-10 from final_summary
    illness_type = final_summary.loc[final_summary['icd10'] == diag['icd10'], 'illness_type'].values[0]

    # Generate order_items
    order_items = []
    matching_order_items = final_order_summary[final_order_summary['icd10'] == diag['icd10']]

    if not matching_order_items.empty:
        for _, order_row in matching_order_items.iterrows():
            # Apply condition to select items with price within ±5% of average_initial_price
            min_price = order_row['average_initial_price'] * 0.95
            max_price = order_row['average_initial_price'] * 1.05
            initial_price = random.uniform(min_price, max_price)

            # Create order item with initial price and other details
            order_items.append({
                'item_id': order_row['item_id'],
                'item_name': order_row['item_name'],
                'simb_billing_code': order_row['simb_billing_code'],
                'initial': f"{initial_price:.2f}",
                'local_billing_name': order_row['local_billing_name']
            })

    # Now calculate billing for each item based on the order_item prices
    billing = []
    for order_item in order_items:
        billing_initial = float(order_item['initial'])  # Use the initial price from order item
        billing_discount = random.uniform(0, billing_initial * 0.1)  # Random discount up to 10%
        billing_net_amount = billing_initial - billing_discount

        billing.append({
            'billing_initial': f"{billing_initial:.2f}",
            'billing_discount': f"{billing_discount:.2f}",
            'simb_billing_code': order_item['simb_billing_code'],
            'billing_net_amount': f"{billing_net_amount:.2f}",
            'local_billing_code': order_item['simb_billing_code'],
            'local_billing_name': order_item['local_billing_name']
        })

    # Calculate total_bill_amount by summing billing_net_amount in billing
    total_bill_amount = sum(float(item['billing_net_amount']) for item in billing)

    # Append the row with all information, including illness_type and total_bill_amount
    data.append({
        'hospital_code': hospital_code,
        'cid': cid,
        'patient': patient,
        'diagnosis': diagnosis,
        'vital_sign': vital_sign,
        'doctor': doctor,
        'billing': billing,
        'order_item': order_items if order_items else [],  # Include only if available
        'illness_type': illness_type,
        'total_bill_amount': f"{total_bill_amount:.2f}"
    })

In [54]:
# Display a sample of the data
mockup_data = pd.DataFrame(data)
display(mockup_data)

,hospital_code,cid,patient,diagnosis,vital_sign,doctor,billing,order_item,illness_type,total_bill_amount
0,10821,3213485831638,"{'hn': '22-24322', 'dob': '1979-07-27', 'gende...","[{'icd10': 'L308', 'dx_name': 'Other specified...","[{'heart_rate': '97', 'pain_score': '0', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '201.81', 'billing_discou...","[{'item_id': '0107H06D0100L01', 'item_name': '...",ILL,13054.24
1,23875,8813788904047,"{'hn': '84-43704', 'dob': '1919-01-01', 'gende...","[{'icd10': 'L210', 'dx_name': 'Seborrhoea capi...","[{'heart_rate': '79', 'pain_score': '6', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '700.86', 'billing_discou...","[{'item_id': '8294', 'item_name': 'SCALP LOTIO...",ILL,888.45
2,11771,7500667210021,"{'hn': '72-33748', 'dob': '1944-10-18', 'gende...","[{'icd10': 'N40', 'dx_name': 'Hyperplasia of p...","[{'heart_rate': '84', 'pain_score': '2', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '264.00', 'billing_discou...","[{'item_id': '0102G03D00', 'item_name': 'DOXAZ...",ILL,7977.08
3,11963,6284596298838,"{'hn': '96-80765', 'dob': '1984-06-11', 'gende...","[{'icd10': 'R119', 'dx_name': 'Nausea and vomi...","[{'heart_rate': '95', 'pain_score': '4', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '186.32', 'billing_discou...","[{'item_id': '0101B05M0010O02', 'item_name': '...",ILL,4315.22
4,41829,8588662232033,"{'hn': '38-64130', 'dob': '1996-08-31', 'gende...","[{'icd10': 'K801', 'dx_name': 'Calculus of gal...","[{'heart_rate': '94', 'pain_score': '4', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '168.74', 'billing_discou...","[{'item_id': 'NSOP', 'item_name': 'บริการพยาบา...",ILL,1009.84
...,...,...,...,...,...,...,...,...,...,...
4995,41556,9698893337367,"{'hn': '87-31332', 'dob': '1979-11-02', 'gende...","[{'icd10': 'B084', 'dx_name': 'Enteroviral ves...","[{'heart_rate': '75', 'pain_score': '5', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '644.43', 'billing_discou...","[{'item_id': '0117A04Z0010O02', 'item_name': '...",ILL,13543.96
4996,10686,1572330995662,"{'hn': '54-68177', 'dob': '2006-09-23', 'gende...","[{'icd10': 'M1007', 'dx_name': 'Idiopathic gou...","[{'heart_rate': '82', 'pain_score': '4', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '250.43', 'billing_discou...","[{'item_id': '0104C08C0006L01', 'item_name': '...",ILL,11017.87
4997,10827,6509552210273,"{'hn': '49-73036', 'dob': '2021-02-09', 'gende...","[{'icd10': 'C220', 'dx_name': 'Liver cell carc...","[{'heart_rate': '80', 'pain_score': '8', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '176.59', 'billing_discou...","[{'item_id': 'NSE', 'item_name': 'ค่าบริการพยา...",ILL,893.74
4998,13779,2619619981272,"{'hn': '44-18819', 'dob': '1955-05-26', 'gende...","[{'icd10': 'S410', 'dx_name': 'Open wound of s...","[{'heart_rate': '74', 'pain_score': '7', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '167.13', 'billing_discou...","[{'item_id': 'DRUG', 'item_name': 'ACEO (Aceme...",ACC,1858.64


In [138]:
import json

# ฟังก์ชันที่ใช้ในการดึงรหัส ICD-10 จาก `diagnosis` ซึ่งเป็นลิสต์
def extract_primary_icd10(diagnosis_list):
    try:
        # หาก diagnosis_list เป็นลิสต์
        if isinstance(diagnosis_list, list):
            for d in diagnosis_list:
                # ตรวจสอบว่าเป็น dictionary และหาค่าของ icd10
                if isinstance(d, dict) and 'icd10' in d:
                    return d.get("icd10")
    except (json.JSONDecodeError, TypeError):
        pass
    return None

# สมมติว่า mockup_data คือ DataFrame ที่มีคอลัมน์ 'diagnosis' ซึ่งเป็นลิสต์
mockup_data['icd10'] = mockup_data['diagnosis'].apply(extract_primary_icd10)

In [140]:
mockup_data[mockup_data['icd10']=='W260']

,hospital_code,cid,patient,diagnosis,vital_sign,doctor,billing,order_item,illness_type,total_bill_amount,icd10
2,11644,7572324496302,"{'hn': '23-65928', 'dob': '1932-08-14', 'gende...","[{'icd10': 'W260', 'dx_name': 'Contact with ot...","[{'heart_rate': '72', 'pain_score': '9', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '140.92', 'billing_discou...","[{'item_id': '0104B04P0500L05', 'item_name': '...",ACC,116076.12,W260
59,11768,2028689523055,"{'hn': '91-60936', 'dob': '1999-11-22', 'gende...","[{'icd10': 'W260', 'dx_name': 'Contact with ot...","[{'heart_rate': '63', 'pain_score': '0', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '133.67', 'billing_discou...","[{'item_id': '0104B04P0500L05', 'item_name': '...",ACC,116421.72,W260
905,10734,6595531964877,"{'hn': '34-38815', 'dob': '1949-11-14', 'gende...","[{'icd10': 'W260', 'dx_name': 'Contact with ot...","[{'heart_rate': '72', 'pain_score': '6', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '132.51', 'billing_discou...","[{'item_id': '0104B04P0500L05', 'item_name': '...",ACC,116447.65,W260
2796,12230,2363462979480,"{'hn': '21-61491', 'dob': '1957-02-09', 'gende...","[{'icd10': 'W260', 'dx_name': 'Contact with ot...","[{'heart_rate': '92', 'pain_score': '2', 'syst...","[{'role': 'OWNER', 'title': 'นพ.', 'license': ...","[{'billing_initial': '135.51', 'billing_discou...","[{'item_id': '0104B04P0500L05', 'item_name': '...",ACC,116348.37,W260


In [145]:
json.loads(df[df['icd10']=='W260'][['billing','total_bill_amount','diagnosis','icd10','order_item']]['billing'].iloc[0])

[{'billing_initial': '70',
  'billing_discount': '0',
  'simb_billing_code': '1.1.2(1)',
  'billing_net_amount': '70.00',
  'local_billing_code': 'SUP1',
  'local_billing_name': 'Medical Supplies 1   เวชภัณฑ์1'},
 {'billing_initial': '300',
  'billing_discount': '0',
  'simb_billing_code': '1.1.14(1)',
  'billing_net_amount': '300.00',
  'local_billing_code': 'DRES',
  'local_billing_name': 'Packaged Medical Charges  ค่าบริการชุดเหมาจ่าย '}]

In [144]:
final_summary[final_summary['icd10']=='W260']

,simb_billing_code,local_billing_name,average_initial_price,billing_count,min_initial_price,max_initial_price,price_list,range,icd10,illness_type
165,1.1.1(2),Home Medication ค่ายากลับบ้าน,85.0,1,85.0,85.0,[85.0],85.0-85.0,W260,ACC
166,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,227.0,113,0.0,3447.0,"[0.0, 19.0, 20.0, 29.0, 30.0, 40.0, 53.0, 66.0...",0.0-3447.0,W260,ACC
167,1.1.1(4),ค่าสารอาหารทางเส้นเลือด,116.0,7,96.0,290.0,"[96.0, 116.0, 290.0]",96.0-290.0,W260,ACC
168,1.1.12,Nursing and Midwifery Charge ค่าบริการพยาบาล,185.0,144,50.0,750.0,"[50.0, 60.0, 75.0, 100.0, 120.0, 140.0, 150.0,...",50.0-750.0,W260,ACC
169,1.1.14(1),Packaged Medical Charges ค่าบริการชุดเหมาจ่าย,200.0,91,100.0,500.0,"[100.0, 130.0, 140.0, 150.0, 170.0, 200.0, 280...",100.0-500.0,W260,ACC
170,1.1.14(2),Other Medical Service Charge ค่าบริการทางการแพทย์,185.0,145,0.0,310.0,"[0.0, 30.0, 40.0, 80.0, 100.0, 120.0, 125.0, 1...",0.0-310.0,W260,ACC
171,1.1.2(1),Medical Supplies 1 เวชภัณฑ์1,231.0,195,0.0,2140.0,"[0.0, 5.0, 6.0, 8.0, 11.0, 15.0, 16.0, 18.0, 1...",0.0-2140.0,W260,ACC
172,1.1.5(1),Standard Diagnostic ค่าตรวจรังสีปกติ รังสี...,1060.0,4,882.0,1418.0,"[882.0, 1060.0, 1418.0]",882.0-1418.0,W260,ACC
173,1.1.7(1),General Medical Equipment ค่าอุปกรณ์นอกห้องผ่...,320.0,40,63.0,1139.0,"[63.0, 100.0, 150.0, 270.0, 290.0, 350.0, 468....",63.0-1139.0,W260,ACC
174,1.1.7(2),Operation Room Medical Equipment ค่าอุปกรณ์ขอ...,162.0,9,130.0,1100.0,"[130.0, 150.0, 162.0, 200.0, 624.0, 800.0, 110...",130.0-1100.0,W260,ACC


In [127]:
final_summary[final_summary['icd10']=='M6001']

,simb_billing_code,local_billing_name,average_initial_price,billing_count,min_initial_price,max_initial_price,price_list,range,icd10,illness_type
15984,1.2.1,Physician Evaluation and Management Services,1000.0,1,1000.0,1000.0,[1000.0],1000.0-1000.0,M6001,ILL


In [81]:
mockup_data[mockup_data['icd10']=='I847']

,hospital_code,cid,patient,diagnosis,vital_sign,doctor,billing,order_item,illness_type,total_bill_amount,icd10


In [ ]:
mockup_data.to_csv('/content/drive/MyDrive/AIEngineer/fraud/mockup/mock1.csv')

#### check 1

In [ ]:
filtered_data = final_summary[final_summary['icd10'] == 'N762']
display(filtered_data)
print(filtered_data['average_initial_price'].sum())

,simb_billing_code,local_billing_name,average_initial_price,billing_count,min_initial_price,max_initial_price,range,icd10,illness_type
13359,1.1.1(3),Outpatient Medication ค่ายาผู้ป่วยนอก,480.0,1,480.0,480.0,480.0-480.0,N762,ILL
13360,1.2.1(1),First Outpatient care before admission ค่าแพท...,600.0,1,600.0,600.0,600.0-600.0,N762,ILL
13361,1.2.2,Physician Procedures ค่าแพทย์ศัลยกรรม หัตถกา...,120.0,1,120.0,120.0,120.0-120.0,N762,ILL


1200.0


In [ ]:
print(mock2['total_bill_amount'].iloc[-2])
print(mock2['diagnosis'].iloc[-2], '\n')
mock2['billing'].iloc[-2]

1118.12
[{'icd10': 'N762', 'dx_name': 'Acute vulvitis'}] 



[{'billing_initial': '480.00',
  'billing_discount': '62.39',
  'simb_billing_code': '1.1.1(3)',
  'billing_net_amount': '417.61',
  'local_billing_code': '1.1.1(3)',
  'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'},
 {'billing_initial': '600.00',
  'billing_discount': '5.13',
  'simb_billing_code': '1.2.1(1)',
  'billing_net_amount': '594.87',
  'local_billing_code': '1.2.1(1)',
  'local_billing_name': 'First Outpatient care before admission  ค่าแพทย์ตรวจรักษาครั้งแรก'},
 {'billing_initial': '120.00',
  'billing_discount': '14.36',
  'simb_billing_code': '1.2.2',
  'billing_net_amount': '105.64',
  'local_billing_code': '1.2.2',
  'local_billing_name': 'Physician Procedures  ค่าแพทย์ศัลยกรรม  หัตถการผ่าตัด'}]